# Hotel Booking Demand - Processing

In [1]:
import pandas as pd
import numpy as np

RANDOM_SEED = 1

/Users/split/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Process dataset for H1

H1 is a resort hotel.

In [2]:
# modify the file path to `H1.csv` if necessary
h1_filepath = './H1.csv'
h1 = pd.read_csv(h1_filepath)
h1.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0,...,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0,...,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
h1.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'ReservationStatus', 'ReservationStatusDate'],
      dtype='object')

## Verify data types for each column

In [4]:
types = []
for column in h1.columns:
    types.append(type(h1[column][0]))
    
col_type = dict(zip(h1.columns, types))
col_type

{'IsCanceled': numpy.int64,
 'LeadTime': numpy.int64,
 'ArrivalDateYear': numpy.int64,
 'ArrivalDateMonth': str,
 'ArrivalDateWeekNumber': numpy.int64,
 'ArrivalDateDayOfMonth': numpy.int64,
 'StaysInWeekendNights': numpy.int64,
 'StaysInWeekNights': numpy.int64,
 'Adults': numpy.int64,
 'Children': numpy.int64,
 'Babies': numpy.int64,
 'Meal': str,
 'Country': str,
 'MarketSegment': str,
 'DistributionChannel': str,
 'IsRepeatedGuest': numpy.int64,
 'PreviousCancellations': numpy.int64,
 'PreviousBookingsNotCanceled': numpy.int64,
 'ReservedRoomType': str,
 'AssignedRoomType': str,
 'BookingChanges': numpy.int64,
 'DepositType': str,
 'Agent': str,
 'Company': str,
 'DaysInWaitingList': numpy.int64,
 'CustomerType': str,
 'ADR': numpy.float64,
 'RequiredCarParkingSpaces': numpy.int64,
 'TotalOfSpecialRequests': numpy.int64,
 'ReservationStatus': str,
 'ReservationStatusDate': str}

In [5]:
# Check if strings are properly formatted 
h1['ReservedRoomType'].unique()

array(['C               ', 'A               ', 'D               ',
       'E               ', 'G               ', 'F               ',
       'H               ', 'L               ', 'P               ',
       'B               '], dtype=object)

In [6]:
# strip trailing whitespace
h1['ReservedRoomType'] = h1['ReservedRoomType'].str.strip()
print(h1['ReservedRoomType'].unique())

# strip whitespace for `AssignedRoomType`
h1['AssignedRoomType'] = h1['AssignedRoomType'].str.strip()

['C' 'A' 'D' 'E' 'G' 'F' 'H' 'L' 'P' 'B']


In [7]:
# strip whitespace for `Agent`
h1['Agent'] = h1['Agent'].str.strip()
print(h1['Agent'].unique()[:10])

# strip whitespace for `Company`, `CustomerType`, `DepositType`
h1['Company'] = h1['Company'].str.strip()
h1['CustomerType'] = h1['CustomerType'].str.strip()
h1['DepositType'] = h1['DepositType'].str.strip()

['NULL' '304' '240' '303' '15' '241' '8' '250' '115' '5']


----
##### (unused) Converting `ArrivalDateMonth` column from string to integer

In [ ]:
# create month dict
month_dict = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

# define month function to apply to column
def month(val):
    return month_dict[val]

In [ ]:
# apply month() to column, reassign to dataframe
h1['ArrivalDateMonth'] = h1['ArrivalDateMonth'].apply(month)

Create new column `ArrivalDate`, which is `ArrivalDateYear`, `ArrivalDateMonth`, `ArrivalDateDayOfMonth` in YYYY-MM-DD format (as in `ReservationStatusDate`).

In [ ]:
def create_arrival_date(s):
    return "{}-{}-{}".format(s['ArrivalDateYear'], s['ArrivalDateMonth'], s['ArrivalDateDayOfMonth'])

arrival_dates = h1.apply(create_arrival_date, axis=1)

In [ ]:
arrival_dates

----

## Check relationship between `IsCanceled` and `ReservationStatus`

In [8]:
h1[['IsCanceled', 'ReservationStatus']].value_counts()

IsCanceled  ReservationStatus
0           Check-Out            28938
1           Canceled             10831
            No-Show                291
dtype: int64

In [9]:
h1[(h1['IsCanceled']==1) & (h1['ReservationStatus'] =='No-Show')]

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
122,1,36,2015,July,27,4,0,1,1,0,...,No Deposit,250,NULL,0,Transient,123.00,0,0,No-Show,2015-07-04
328,1,55,2015,July,29,12,4,10,2,10,...,No Deposit,8,NULL,0,Contract,133.16,0,1,No-Show,2015-07-12
343,1,47,2015,July,29,12,2,3,2,0,...,No Deposit,241,NULL,0,Transient,103.80,0,0,No-Show,2015-07-12
467,1,6,2015,July,29,17,0,2,2,0,...,No Deposit,241,NULL,0,Transient,110.50,0,1,No-Show,2015-07-17
472,1,3,2015,July,29,17,0,2,1,0,...,No Deposit,5,NULL,0,Transient,107.60,0,0,No-Show,2015-07-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28550,1,10,2016,October,41,8,0,0,2,0,...,No Deposit,NULL,405,0,Transient-Party,0.00,0,1,No-Show,2016-10-08
29537,1,1,2016,October,45,30,1,0,2,0,...,No Deposit,NULL,NULL,0,Transient,55.00,0,0,No-Show,2016-10-30
29694,1,3,2017,January,3,19,0,1,1,0,...,No Deposit,NULL,470,0,Transient,35.00,0,1,No-Show,2017-01-19
32349,1,0,2017,February,7,17,0,1,1,0,...,No Deposit,NULL,195,0,Transient,35.00,0,0,No-Show,2017-02-17


In [10]:
291 / h1.shape[0]

0.007264103844233649

### Removing No-Shows

Because less than 1% of customers fail to show up in H1, no-shows will be removed from the dataset.

In [11]:
h1 = h1[h1['ReservationStatus'] !='No-Show']

## Remove `Country` column

In [12]:
h1 = h1.drop('Country', axis=1)

## Examine columns related to travel agencies, tour operators, booking channels 

A **hotel distribution channel** can be any method or platform by which your hotel sells its rooms. Examples include online travel agents, booking engines (website + social media), phone/email/walk-ins, metasearch, global distribution system.

In [13]:
h1['DistributionChannel'].value_counts()

TA/TO        28740
Direct        7798
Corporate     3230
Undefined        1
Name: DistributionChannel, dtype: int64

In [14]:
# Remove rows where `DistributionChannel` is "Undefined" 
h1 = h1[h1['DistributionChannel'] != 'Undefined']

In [15]:
h1ta = h1[['MarketSegment', 'DepositType', 'Agent', 'Company', 'CustomerType']]
h1ta.head()

,MarketSegment,DepositType,Agent,Company,CustomerType
0,Direct,No Deposit,NULL,NULL,Transient
1,Direct,No Deposit,NULL,NULL,Transient
2,Direct,No Deposit,NULL,NULL,Transient
3,Corporate,No Deposit,304,NULL,Transient
4,Online TA,No Deposit,240,NULL,Transient


In [16]:
print(len(h1ta['Company'].unique()))

h1ta['Company'].value_counts()

236


NULL    36700
223       777
281       135
154       133
405        97
        ...  
401         1
377         1
109         1
365         1
410         1
Name: Company, Length: 236, dtype: int64

In [17]:
print(len(h1ta['Agent'].unique()))

h1ta['Agent'].value_counts()

186


240     13804
NULL     8134
250      2837
241      1714
40        996
        ...  
406         1
333         1
431         1
433         1
187         1
Name: Agent, Length: 186, dtype: int64

Since company and agent are categorical variables and some agencies/companies appear a few times throughout the dataset, these two variables  can be converted to binary - 1 if it was booked through an agency/company, 0 if not. 

Additional note: in each dataset, agent/company is "incomplete" (i.e. we see company #410 but only 236 unique companies are present in H1) since each dataset is compiled from a database -- companies that don't appear in H1 can appear in H2, vice versa (the same applies for `agent`).

In [18]:
h1['Agent'] = np.where(h1['Agent']=='NULL', 0, 1)
h1['Company'] =  np.where(h1['Company']=='NULL', 0, 1)

In [19]:
# one-hot-encoding `DistributionChannel`, dropping original column along with `MarketSegment`
h1 = h1.join(pd.get_dummies(h1['DistributionChannel']))
h1 = h1.drop(columns=['DistributionChannel', 'MarketSegment'])

## Calculating % of bookings that had a deposit

Converting `DepositType` into a binary variable, `Deposit`, and dropping the original column.

In [20]:
dt_vc = h1ta['DepositType'].value_counts()
(dt_vc[1] + dt_vc[2]) / h1.shape[0]

0.04631864815932408

In [21]:
h1[['IsCanceled', 'DepositType']].value_counts(normalize=True)

IsCanceled  DepositType
0           No Deposit     0.722893
1           No Deposit     0.230789
            Non Refund     0.041038
0           Refundable     0.003018
            Non Refund     0.001735
1           Refundable     0.000528
dtype: float64

In [22]:
h1[['DepositType', 'Agent']].value_counts(normalize=True)

DepositType  Agent
No Deposit   1        0.764409
             0        0.189273
Non Refund   1        0.030301
             0        0.012472
Refundable   0        0.002791
             1        0.000754
dtype: float64

In [23]:
h1[['DepositType', 'Company']].value_counts(normalize=True)

DepositType  Company
No Deposit   0          0.882141
             1          0.071540
Non Refund   0          0.039479
             1          0.003294
Refundable   1          0.002313
             0          0.001232
dtype: float64

In [24]:
deposit_ = {
    'No Deposit': 0,
    'Non Refund': 1, 
    'Refundable': 1
}

def encode_deposit_type(entry):
    return deposit_[entry]

h1['Deposit'] = h1['DepositType'].str.strip().apply(encode_deposit_type)
h1 = h1.drop(columns=['DepositType'])

----
##   Encode `Meal` column

- Undefined/SC – no meal package;
- BB – Bed & Breakfast;
- HB – Half board (breakfast and one other meal – usually dinner);
- FB – Full board (breakfast, lunch and dinner)

Convert to binary variable: 1 if there is a meal package (BB, HB, FB), 0 otherwise

In [25]:
h1['Meal'].value_counts()

BB           29754
HB            8012
Undefined     1167
FB             749
SC              86
Name: Meal, dtype: int64

In [26]:
meals = {
    'BB': 1,
    'HB': 1,
    'FB': 1,
    'SC': 0,
    'Undefined': 0
}

def encode_meals(s):
    return meals[s]

h1['Meal'] = h1['Meal'].str.strip().apply(encode_meals)

----
## Creating new variable based on room assignment

`AssignedRoomType` and `ReservedRoomType` are encoded for anonymity. It may be helpful to look at the types of rooms, if requested but not assigned (and also the other way around), that will cause people to cancel their bookings. With this in mind, we can combine these two columns to create binary variable - whether the assigned room is different from the reserved room.

Note: According to the metadata, customers may request to change their room type (under `BookingChanges`), but we cannot tell if the room type differences is due to overbooking or by request because the room types are coded.

In [27]:
h1[['AssignedRoomType', 'ReservedRoomType']]

,AssignedRoomType,ReservedRoomType
0,C,C
1,C,C
2,C,A
3,A,A
4,A,A
...,...,...
40055,A,A
40056,E,E
40057,E,E
40058,D,D


In [28]:
h1['RoomTypeDifference'] = np.where(h1['AssignedRoomType'] == h1['ReservedRoomType'], 0, 1)

# Drop `AssignedRoomType` and `ReservedRoomType`
h1 = h1.drop(columns=['AssignedRoomType', 'ReservedRoomType'])

-----

In [29]:
# one-hot-encode `CustomerType`, drop original column
h1 = h1.join(pd.get_dummies(h1['CustomerType']))
h1 = h1.drop(columns=['CustomerType'])

In [30]:
# Drop `ReservationStatus` -- an exact copy of `IsCanceled` in string vs. binary
# (drop the status date together)
h1 = h1.drop(columns=['ReservationStatus', 'ReservationStatusDate'])

In [31]:
# check for any NaNs
for i in h1.columns:
    if any(h1[i][h1[i].isna()]):
           print(i)

### Truncating dataset size 

Since the dataset has almost 40000 rows, random sampling will be performed to reduce the size. `n` can be modified below.

In [32]:
n = 4000

h1 = h1.sample(n, random_state=RANDOM_SEED)

In [33]:
print(h1.shape)
h1.columns

(4000, 31)


Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'Agent', 'Company',
       'DaysInWaitingList', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'Corporate', 'Direct', 'TA/TO', 'Deposit',
       'RoomTypeDifference', 'Contract', 'Group', 'Transient',
       'Transient-Party'],
      dtype='object')

In [34]:
# Save to csv
h1.to_csv('h1_s.csv', index=False)

------

# Processing for H2

Performing the exact same methods on H1, but this time on H2, a city hotel.

In [35]:
h2_filepath = './H2.csv'
h2 = pd.read_csv(h2_filepath)
h2.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [36]:
types = []
for column in h2.columns:
    types.append(type(h2[column][0]))
    
col_type = dict(zip(h2.columns, types))
col_type

{'IsCanceled': numpy.int64,
 'LeadTime': numpy.int64,
 'ArrivalDateYear': numpy.int64,
 'ArrivalDateMonth': str,
 'ArrivalDateWeekNumber': numpy.int64,
 'ArrivalDateDayOfMonth': numpy.int64,
 'StaysInWeekendNights': numpy.int64,
 'StaysInWeekNights': numpy.int64,
 'Adults': numpy.int64,
 'Children': numpy.float64,
 'Babies': numpy.int64,
 'Meal': str,
 'Country': str,
 'MarketSegment': str,
 'DistributionChannel': str,
 'IsRepeatedGuest': numpy.int64,
 'PreviousCancellations': numpy.int64,
 'PreviousBookingsNotCanceled': numpy.int64,
 'ReservedRoomType': str,
 'AssignedRoomType': str,
 'BookingChanges': numpy.int64,
 'DepositType': str,
 'Agent': str,
 'Company': str,
 'DaysInWaitingList': numpy.int64,
 'CustomerType': str,
 'ADR': numpy.float64,
 'RequiredCarParkingSpaces': numpy.int64,
 'TotalOfSpecialRequests': numpy.int64,
 'ReservationStatus': str,
 'ReservationStatusDate': str}

In [37]:
# stripping whitespace
h2['ReservedRoomType'] = h2['ReservedRoomType'].str.strip()

h2['AssignedRoomType'] = h2['AssignedRoomType'].str.strip()

h2['Agent'] = h2['Agent'].str.strip()

h2['Company'] = h2['Company'].str.strip()

h2['CustomerType'] = h2['CustomerType'].str.strip()

h2['DepositType'] = h2['DepositType'].str.strip()

In [38]:
h2[['IsCanceled', 'ReservationStatus']].value_counts()

IsCanceled  ReservationStatus
0           Check-Out            46228
1           Canceled             32186
            No-Show                916
dtype: int64

In [39]:
916 / h2.shape[0]

0.011546703643010211

In [40]:
# dropping no-shows
h2 = h2[h2['ReservationStatus'] != 'No-Show']

In [41]:
# drop `country`
h2 = h2.drop('Country', axis=1)

In [42]:
# Remove rows where `DistributionChannel` is "Undefined" 
h2 = h2[h2['DistributionChannel'] != 'Undefined']

# convert `agent` and `company` into a binary variable
h2['Agent'] = np.where(h2['Agent']=='NULL', 0, 1)
h2['Company'] = np.where(h2['Company']=='NULL', 0, 1)

# one-hot-encoding `DistributionChannel`, dropping original column along with `MarketSegment`
h2 = h2.join(pd.get_dummies(h2['DistributionChannel']))
h2 = h2.drop(columns=['DistributionChannel', 'MarketSegment'])

In [43]:
# convert `DepositType` into a binary variable
h2['Deposit'] = h2['DepositType'].str.strip().apply(encode_deposit_type)
h2 = h2.drop(columns=['DepositType'])

In [44]:
# convert `meal` into a binary variable using previously defined function
h2['Meal'] = h2['Meal'].str.strip().apply(encode_meals)

In [45]:
# create binary variable if assigned and reserved room type is different
h2['RoomTypeDifference'] = np.where(h2['AssignedRoomType'] == h2['ReservedRoomType'], 0, 1)

h2 = h2.drop(columns=['AssignedRoomType', 'ReservedRoomType'])

In [46]:
# one-hot-encode `CustomerType`
h2 = h2.join(pd.get_dummies(h2['CustomerType']))

# Drop `CustomerType`, reservation status
h2 = h2.drop(columns=['CustomerType', 'ReservationStatus', 'ReservationStatusDate'])

In [47]:
# check for any missing values
for i in h2.columns:
    if any(h2[i][h2[i].isna()]):
           print(i)

In [48]:
h2['Children'][h2['Children'].isna()]

Series([], Name: Children, dtype: float64)

In [49]:
h2.shape

(78410, 32)

In [50]:
# since 4 our of ~78.4k records are nan, those rows will be dropped
h2 = h2[~(h2['Children'].isna())]

In [51]:
# Reduce dataset size by sampling
n = 4000

h2 = h2.sample(n, random_state=RANDOM_SEED)

In [52]:
print(h2.shape)
h2.columns

(4000, 32)


Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'Agent', 'Company',
       'DaysInWaitingList', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'Corporate', 'Direct', 'GDS', 'TA/TO',
       'Deposit', 'RoomTypeDifference', 'Contract', 'Group', 'Transient',
       'Transient-Party'],
      dtype='object')

In [53]:
# save to csv
h2.to_csv('h2_s.csv', index=False)

---